In [40]:
import requests
import json
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from pandas import DataFrame, Series
from pandas.io.json import json_normalize
import pprint
import simplejson as json
import bokeh
key = "DW9RJMI2ELO8SDLK"
from datetime import datetime

In [41]:
ts = TimeSeries(key=key)
# Get json object with the intraday data and another with  the call's metadata
data, meta_data = ts.get_intraday('GOOGL')
meta_data

{'1. Information': 'Intraday (15min) open, high, low, close prices and volume',
 '2. Symbol': 'GOOGL',
 '3. Last Refreshed': '2020-06-19 16:00:00',
 '4. Interval': '15min',
 '5. Output Size': 'Compact',
 '6. Time Zone': 'US/Eastern'}

In [153]:
import urllib3
ticker = 'AAPL'
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}'.format(ticker, key)
http = urllib3.PoolManager()
response = http.request('GET', url)
print(response.status) # 200 means the request has succeeded


200


In [154]:
ticker = 'AAPL'

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}'.format(ticker, key)
r = requests.get(url)
# print(r.status_code)
# pprint.pprint(r.json())
json_response = r.json()
# df = pd.DataFrame(json_response['Time Series (Daily)']).T
df = pd.DataFrame.from_dict(json_response['Time Series (Daily)'], orient='index')
df = df.reset_index()
df

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
0,2020-06-19,354.6350,356.5600,345.1500,349.7200,349.7200,64107713,0.0000,1.0000
1,2020-06-18,351.4100,353.4500,349.2200,351.7300,351.7300,24205096,0.0000,1.0000
2,2020-06-17,355.1500,355.4000,351.0900,351.5900,351.5900,28601626,0.0000,1.0000
3,2020-06-16,351.4600,353.2000,344.7200,352.0800,352.0800,41357182,0.0000,1.0000
4,2020-06-15,333.2500,345.6800,332.5800,342.9900,342.9900,34702230,0.0000,1.0000
...,...,...,...,...,...,...,...,...,...
95,2020-02-04,315.3100,319.6400,313.6345,318.8500,317.2459,34154134,0.0000,1.0000
96,2020-02-03,304.3000,313.4900,302.2200,308.6600,307.1071,43496401,0.0000,1.0000
97,2020-01-31,320.9300,322.6800,308.2900,309.5100,307.9529,49897096,0.0000,1.0000
98,2020-01-30,320.5435,324.0900,318.7500,323.8700,322.2406,31685808,0.0000,1.0000


In [155]:
# rename the columns
df = df.rename(index=str, columns={"index": "date", "1. open": "open","2. high": "high", "3. low": "low", "4. close": "close", "5. adjusted close": "adjusted_close", "6. volume": "volume", "7. dividend amount": "dividend_amount", "8. split coefficient": "split_coefficient"})
df = df.drop(columns = ['split_coefficient'])
#renamed
df.columns.tolist()

['date',
 'open',
 'high',
 'low',
 'close',
 'adjusted_close',
 'volume',
 'dividend_amount']

In [156]:
# change to datetime
df['date'] = pd.to_datetime(df['date'])

#Sort data according to date
df = df.sort_values(by=['date'])

In [157]:
for i in list(df.columns):
        if i != 'date':
            df[i] = df[i].astype('float')

In [158]:
df.head()

,date,open,high,low,close,adjusted_close,volume,dividend_amount
99,2020-01-29,324.4500,327.85,321.3800,324.34,322.7082,54149928.0,0.0
98,2020-01-30,320.5435,324.09,318.7500,323.87,322.2406,31685808.0,0.0
97,2020-01-31,320.9300,322.68,308.2900,309.51,307.9529,49897096.0,0.0
96,2020-02-03,304.3000,313.49,302.2200,308.66,307.1071,43496401.0,0.0
95,2020-02-04,315.3100,319.64,313.6345,318.85,317.2459,34154134.0,0.0


In [159]:
df['daily_return'] = (df['close'] - df['open'] + df['dividend_amount'])/df['open']
df

,date,open,high,low,close,adjusted_close,volume,dividend_amount,daily_return
99,2020-01-29,324.4500,327.85,321.3800,324.34,322.7082,54149928.0,0.0,-0.000339
98,2020-01-30,320.5435,324.09,318.7500,323.87,322.2406,31685808.0,0.0,0.010378
97,2020-01-31,320.9300,322.68,308.2900,309.51,307.9529,49897096.0,0.0,-0.035584
96,2020-02-03,304.3000,313.49,302.2200,308.66,307.1071,43496401.0,0.0,0.014328
95,2020-02-04,315.3100,319.64,313.6345,318.85,317.2459,34154134.0,0.0,0.011227
...,...,...,...,...,...,...,...,...,...
4,2020-06-15,333.2500,345.68,332.5800,342.99,342.9900,34702230.0,0.0,0.029227
3,2020-06-16,351.4600,353.20,344.7200,352.08,352.0800,41357182.0,0.0,0.001764
2,2020-06-17,355.1500,355.40,351.0900,351.59,351.5900,28601626.0,0.0,-0.010024
1,2020-06-18,351.4100,353.45,349.2200,351.73,351.7300,24205096.0,0.0,0.000911


In [160]:
import numpy as np
def datetime(x):
    return np.array(x, dtype=np.datetime64)

In [144]:
# datetime(df["date_time"])

In [50]:
df.columns

Index(['open', 'high', 'low', 'close', 'adjusted_close', 'volume',
       'dividend_amount', 'date_time', 'return_of_the_day'],
      dtype='object')

In [147]:
df.date

99   2020-01-29
98   2020-01-30
97   2020-01-31
96   2020-02-03
95   2020-02-04
        ...    
4    2020-06-15
3    2020-06-16
2    2020-06-17
1    2020-06-18
0    2020-06-19
Name: date, Length: 100, dtype: datetime64[ns]

In [52]:
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.plotting import figure, output_file, show
#link Pandas’ DataFrame with Bokeh visualizations
# Bokeh Object ColumnDataSource accepts DataFrame, pass to glyph 
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models import DatetimeTickFormatter
import datetime as dt
import pandas_bokeh

#pandas_bokeh.output_notebook()


'\n#output_file(\'put.html\')\np = figure(title=ticker, x_axis_type = \'datetime\', plot_height=600, plot_width=800)\np.title.text_font_size = "30px"\np.xaxis.axis_label = \'Time\'\np.yaxis.axis_label = \'Price\'\n\np.line(x=df["date_time"], y=df[\'close\'])\n# p.xaxis.formatter = DatetimeTickFormatter(days=["%b/%d/%Y"])\n# p.xaxis.minor_tick_line_color = None\n# p.xaxis.major_label_overrides = {i: df[\'date_time\'].iloc[i].strftime(\'%b %d %Y\') for i in range(len(df))}\n\n# script contains data for th plot, the plot view is loaded into <div> \n#3 script, div = components(p)\nshow(p)\n\n\n\n'

In [53]:
import bokeh
from bokeh.embed import components
from bokeh.plotting import figure, output_file, show
#link Pandas’ DataFrame with Bokeh visualizations
# Bokeh Object ColumnDataSource accepts DataFrame, pass to glyph 
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

In [148]:
np.array(df['close'])

array([324.34, 323.87, 309.51, 308.66, 318.85, 321.45, 325.21, 320.03,
       321.55, 319.61, 327.2 , 324.87, 324.95, 319.  , 323.62, 320.3 ,
       313.05, 298.18, 288.08, 292.65, 273.52, 273.36, 298.81, 289.32,
       302.74, 292.92, 289.03, 266.17, 285.34, 275.43, 248.23, 277.97,
       242.21, 252.86, 246.67, 244.78, 229.24, 224.37, 246.88, 245.52,
       258.44, 247.74, 254.81, 254.29, 240.91, 244.93, 241.41, 262.47,
       259.43, 266.07, 267.99, 273.25, 287.05, 284.43, 286.69, 282.8 ,
       276.93, 268.37, 276.1 , 275.03, 282.97, 283.17, 278.58, 287.73,
       293.8 , 289.07, 293.16, 297.56, 300.63, 303.74, 310.13, 315.01,
       311.41, 307.65, 309.54, 307.71, 314.96, 313.14, 319.23, 316.85,
       318.89, 316.73, 318.11, 318.25, 317.94, 321.85, 323.34, 325.12,
       322.32, 331.5 , 333.46, 343.99, 352.84, 335.9 , 338.8 , 342.99,
       352.08, 351.59, 351.73, 349.72])

In [149]:

from datetime import datetime
select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset', 'redo','save']

dts = df['date'].to_string()
print(dts)
source = ColumnDataSource(data ={
    'date': [datetime.strptime(x, '%Y-%m-%d') for x in dts],
    'price': df['close'],
    'volume': df['volume']
})

p = figure(x_axis_type = "datetime", 
      x_axis_label = 'Time',
      y_axis_label = 'Price (USD)',
      plot_height = 600, plot_width = 800,
      toolbar_location = 'below',
      tools = select_tools)

p.title.align = "center"
p.title.text_font_size = "20px"
p.grid.grid_line_color="white"
p.background_fill_color="#f5f5f5"
p.axis.axis_line_color = None
p.line(x='date', y = 'price', source = source, line_width=2, line_color = 'orange')

hover_tool = HoverTool(tooltips = [
            ('Date', '@date{%F}'),
            ('Stock Price', "@price"),
            ('Volume', "@volume"),
            ],
            formatters = {
            '@date': 'datetime',
            '@price': 'printf',
            '@volume': 'printf',
            },
            mode = 'vline')
p.add_tools(hover_tool) #, formatters, mode='vline'))

99   2020-01-29
98   2020-01-30
97   2020-01-31
96   2020-02-03
95   2020-02-04
94   2020-02-05
93   2020-02-06
92   2020-02-07
91   2020-02-10
90   2020-02-11
89   2020-02-12
88   2020-02-13
87   2020-02-14
86   2020-02-18
85   2020-02-19
84   2020-02-20
83   2020-02-21
82   2020-02-24
81   2020-02-25
80   2020-02-26
79   2020-02-27
78   2020-02-28
77   2020-03-02
76   2020-03-03
75   2020-03-04
74   2020-03-05
73   2020-03-06
72   2020-03-09
71   2020-03-10
70   2020-03-11
69   2020-03-12
68   2020-03-13
67   2020-03-16
66   2020-03-17
65   2020-03-18
64   2020-03-19
63   2020-03-20
62   2020-03-23
61   2020-03-24
60   2020-03-25
59   2020-03-26
58   2020-03-27
57   2020-03-30
56   2020-03-31
55   2020-04-01
54   2020-04-02
53   2020-04-03
52   2020-04-06
51   2020-04-07
50   2020-04-08
49   2020-04-09
48   2020-04-13
47   2020-04-14
46   2020-04-15
45   2020-04-16
44   2020-04-17
43   2020-04-20
42   2020-04-21
41   2020-04-22
40   2020-04-23
39   2020-04-24
38   2020-04-27
37   202

ValueError: time data '9' does not match format '%Y-%m-%d'

In [150]:
show(p)

In [152]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}'.format(ticker, key)
r = requests.get(url)
# print(r.status_code) # 200 (succeeded)
# pprint.pprint(r.json())
# normalize semi-structure JSON data into a flat table
json_response = r.json()

# df = pd.DataFrame(json_response['Time Series (Daily)']).T
df = pd.DataFrame.from_dict(json_response['Time Series (Daily)'], orient = 'index')
df = df.reset_index()
df = df.rename(index=str, columns={"index": "date", "1. open": "open","2. high": "high", "3. low": "low", "4. close": "close", "5. adjusted close": "adjusted_close", "6. volume": "volume", "7. dividend amount": "dividend_amount", "8. split coefficient": "split_coefficient"})
df = df.drop(columns = ['split_coefficient'])

# change to datetime
df['date'] = pd.to_datetime(df['date'])

#Sort data according to date
df = df.sort_values(by=['date'])

# total stock return = [(Ending stock price - Initial stock price) + Dividend]/Initial_stock_price
df['daily_return'] = (df['close'] - df['open'] + df['dividend_amount'])/df['open']

# Convert to float type
for i in list(df.columns):
    if i != 'date':
        df[i] = df[i].astype('float')

TypeError: unsupported operand type(s) for -: 'str' and 'str'